In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

In [2]:
import shared_utils
import pandas as pd
from siuba import *

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [3]:
shared_utils.gtfs_utils.get_trips?

Signature:
shared_utils.gtfs_utils.get_trips(
    selected_date: str | datetime.date = datetime.date(2022, 11, 6),
    itp_id_list: list[int] = None,
    trip_cols: list[str] = None,
    get_df: bool = True,
    custom_filtering: dict = None,
) -> pandas.core.frame.DataFrame | siuba.sql.backend.LazyTbl
Docstring: <no docstring>
File:      ~/data-analyses/_shared_utils/shared_utils/gtfs_utils.py
Type:      function


In [4]:
bbb_trips = shared_utils.gtfs_utils.get_trips(itp_id_list=[300])

In [5]:
bbb_trips >> head(3)

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,...,block_id,shape_id,calitp_extracted_at,calitp_hash,wheelchair_accessible,calitp_deleted_at,bikes_allowed,trip_short_name,direction_id,trip_headsign
0,7140288912325048998,7379196416545803288,886650,3512,300,0,2,2022-11-06,1,2022-08-14,...,None,25997,2022-09-01,3gowVYFHH4OxF4bSeRPitw==,0,2099-01-01,0,None,0,Pacific Palisades
1,7140288912325048998,-6855852687317493983,885050,3504,300,0,2,2022-11-06,1,2022-08-14,...,None,25968,2022-09-01,KwFPq6M14RFc1+WaNHgJWQ==,0,2099-01-01,0,None,0,UCLA
2,7140288912325048998,-1732398069853145487,887667,3519,300,0,2,2022-11-06,1,2022-08-14,...,None,26028,2022-09-01,1hUFlQo3ptAsW9z4DgixVQ==,0,2099-01-01,0,None,1,Marina del Rey


In [6]:
bbb_routes = shared_utils.gtfs_utils.get_route_info(itp_id_list=[300])

In [7]:
bbb_routes >> head(3)

,feed_key,route_key,date,route_sort_order,route_short_name,calitp_extracted_at,agency_email,calitp_itp_id,calitp_deleted_at,route_desc,...,route_color,agency_name,agency_fare_url,route_url,route_type,agency_url,route_continuous_drop_off,agency_id,agency_lang,agency_timezone
0,7140288912325048998,3033573737460649383,2022-11-06,None,18,2022-07-18,None,300,2099-01-01,None,...,76174D,Big Blue Bus,None,http://bigbluebus.com/Routes-and-Schedules/Rou...,3,http://www.bigbluebus.com,None,6216179,en,America/Los_Angeles
1,7140288912325048998,-3537254254914116456,2022-11-06,None,15,2022-07-18,None,300,2099-01-01,None,...,C1D82F,Big Blue Bus,None,http://bigbluebus.com/Routes-and-Schedules/Rou...,3,http://www.bigbluebus.com,None,6216179,en,America/Los_Angeles
2,7140288912325048998,2153019816762057612,2022-11-06,None,41,2022-07-18,None,300,2099-01-01,None,...,E11B22,Big Blue Bus,None,http://bigbluebus.com/Routes-and-Schedules/Rou...,3,http://www.bigbluebus.com,None,6216179,en,America/Los_Angeles


In [8]:
with_names = bbb_trips >> inner_join(_, bbb_routes >> select(_.route_id, _.route_short_name), on = 'route_id')

In [9]:
daily_trips = with_names >> count(_.route_short_name)
daily_trips

,route_short_name,n
0,1,116
1,14,57
2,17,50
3,18,42
4,2,68
5,3,129
6,7,125
7,8,66
8,9,38
9,R12,79


In [10]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In [11]:
route_selector = widgets.Dropdown(options = daily_trips.route_short_name,
                description = "Select Route", style= {'description_width': 'initial'})

In [12]:
increase_box = widgets.IntText(value=0, description='Additional Daily Trips', style= {'description_width': 'initial'})

In [13]:
def show_trips(route, additional_trips):
    current_trips = (daily_trips >> filter(_.route_short_name == route)).n.iloc[0]
    new_trips = current_trips + additional_trips
    # these are made up numbers
    base_factor = 50
    rider_factor = base_factor + additional_trips / 5
    
    current_ridership_est = current_trips*base_factor
    new_ridership_est = new_trips*rider_factor
    display(f'Current daily trips: {current_trips}')
    display(f'Trips after increase: {new_trips}, {new_ridership_est - current_ridership_est} new riders')

In [14]:
interact(show_trips, route=route_selector, additional_trips=increase_box);

interactive(children=(Dropdown(description='Select Route', options=('1', '14', '17', '18', '2', '3', '7', '8',…